# bimbe di bastianin

In [1]:
import pandas as pd
import time
import datetime

ad = pd.read_csv('datasets/1976-2020-president.csv')
years = set(ad['year'])
states = set(ad['state'])

#### some function

In [2]:
def to_percent(older, col_key):
    newer = []
    for y in years:
        for s in states:
            try:
                d = older.loc[y, s, 'DEMOCRAT'][col_key]
                r = older.loc[y, s, 'REPUBLICAN'][col_key]
                newer.append([int(y), s, 100*d/(d+r)]) # % share for each state
            except KeyError as e:
                pass
    return pd.DataFrame(data=newer, columns=['year', 'state', col_key, ])

In [3]:
def to_splatter(a, col_name): # from row-column to table-list
    lst = []
    for year in a.columns[2:]:
        for index, row in a.iterrows():
            lst.append([int(year), row['GeoName'].upper(), row[year], ])
    
    return pd.DataFrame(data=lst, columns=['year', 'state', col_name, ])

#### y_hat aka presitdent popular vote

In [4]:
pop_vote = pd.read_csv('datasets/1976-2020-president.csv')
pop_vote = pop_vote[['year', 'state', 'party_simplified', 'candidatevotes', ]] # drop useless columns
pop_vote = pop_vote.groupby(['year', 'state', 'party_simplified', ]).sum() # drop multiple candidate
pop_vote = to_percent(pop_vote, 'candidatevotes')
pop_vote = pop_vote.rename(columns={"candidatevotes": "y_votes_percent", })
pop_vote.to_csv('datasets-clean/pop_vote.csv', index=False)
pop_vote.head()

,year,state,y_votes_percent
0,1984,NEBRASKA,28.995246
1,1984,COLORADO,35.634230
2,1984,ALABAMA,38.736605
3,1984,MICHIGAN,40.453675
4,1984,MARYLAND,47.242473


#### GDP and friends

It was necessary to generate an adjusted series of state GDP because of a change in BEA’s estimation procedure from a Standard Industrial Classification (SIC) basis to a North American Industry Classification System (NAICS) basis in 1997.
Data prior to 1997 were adjusted to avoid any erratic shifts in GDP that year.
While the change to NAICS basis occurred in 1997, BEA also provides estimates under a SIC basis in that year.
Our adjustment involved calculating the 1997 ratio of NAICS-based GDP to SIC-based GDP for each state, and multiplying it by SIC-based GDP in all years prior to 1997 to obtain our adjusted series of state-level GDP.

In [5]:
gdp_nom_97 = to_splatter(pd.read_csv('datasets/gdp-nominal-63-97.csv'), 'nominal_gdp_mln')
gdp_nom_12 = to_splatter(pd.read_csv('datasets/gdp-nominal-97-20.csv'), 'nominal_gdp_mln')
gdp_real_97 = to_splatter(pd.read_csv('datasets/gdp-real-77-97-chain-97.csv'), 'real_gdp_mln')
gdp_real_12 = to_splatter(pd.read_csv('datasets/gdp-real-97-20-chain-12.csv'), 'real_gdp_mln')

In [6]:
gdp_nom_97 = gdp_nom_97.set_index(['year', 'state'])
gdp_nom_12 = gdp_nom_12.set_index(['year', 'state'])
gdp_real_97 = gdp_real_97.set_index(['year', 'state'])
gdp_real_12 = gdp_real_12.set_index(['year', 'state'])

In [7]:
gdp_def_97 = (gdp_nom_97['nominal_gdp_mln'] / gdp_real_97['real_gdp_mln']).to_frame('gdp_def').dropna()
gdp_def_12 = (gdp_nom_12['nominal_gdp_mln'] / gdp_real_12['real_gdp_mln']).to_frame('gdp_def').dropna()

In [8]:
gdp_def = []

for s in states:
    tmp = gdp_def_97.loc[(slice(None),s),:]
    years = list({i[0] for i in tmp.index})
    years.sort()
    for y in years[1:]:
        q = gdp_def_97.loc[(y,s),:]['gdp_def']/gdp_def_97.loc[(int(y)-1,s),:]['gdp_def']
        gdp_def.append([y, s, (q - 1)*100])

for s in states:
    tmp = gdp_def_12.loc[(slice(None),s),:]
    years = list({i[0] for i in tmp.index})
    years.sort()
    for y in years[1:]:
        q = gdp_def_12.loc[(y,s),:]['gdp_def']/gdp_def_12.loc[(int(y)-1,s),:]['gdp_def']
        gdp_def.append([y, s, (q - 1)*100])

gdp_def = pd.DataFrame(data=gdp_def, columns=['year', 'state', 'gdp_def_percent_change_year_ago'])
gdp_def.head()
gdp_def.to_csv('datasets-clean/gdp-def-percent-change-year-ago.csv', index=False)

#### house

In [9]:
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes')
house_vote['candidatevotes'] = house_vote['candidatevotes'].apply(lambda x: 1 if x < 50 else -1)
house_vote['year'] -= 4
house_vote = house_vote.rename(columns={'candidatevotes':'incumbent'})
house_vote.to_csv('datasets-clean/incumbent-house-rep.csv', index=False)
house_vote.head()

,year,state,incumbent
0,1994,NEBRASKA,1
1,1994,COLORADO,1
2,1994,ALABAMA,1
3,1994,MICHIGAN,-1
4,1994,MARYLAND,-1


In [10]:
'''
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes')
house_vote = house_vote.rename(columns={"candidatevotes": "houserep_votes_percent", })
#'''

'\nhouse_vote = pd.read_csv(\'datasets/1976-2020-house-utf8.csv\')\nhouse_vote = house_vote[[\'year\', \'state\', \'party\', \'candidatevotes\', ]]\nhouse_vote = house_vote.groupby([\'year\', \'state\', \'party\', ]).sum()\nhouse_vote = to_percent(house_vote, \'candidatevotes\')\nhouse_vote = house_vote.rename(columns={"candidatevotes": "houserep_votes_percent", })\n#'

#### incumbent

In [19]:
incumbent = pd.read_csv('datasets/incumbent-4president-76-20.csv', sep=';')
incumbent.head()

tmp = pd.DataFrame(data=[[y,s] for y in years for s in states], columns=['year', 'state'])
incumbent = pd.merge(tmp, incumbent, how='inner', left_on='year', right_on='year')
incumbent.head()

,year,state,incumbent,has_runned_again,more_than_2_terms,sudden_vice,lag_vice
0,2000,NEBRASKA,1,0,0,1,0
1,2000,COLORADO,1,0,0,1,0
2,2000,ALABAMA,1,0,0,1,0
3,2000,MICHIGAN,1,0,0,1,0
4,2000,MARYLAND,1,0,0,1,0


#### gasoline and friends + approval

In [14]:
gas = pd.read_csv('datasets/gasoline-93-21.csv')
gas.date = gas.date.apply(lambda x: time.mktime(datetime.datetime.strptime(x,"%m/%d/%Y").timetuple()))
gas.date = gas.date.apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
gas.head()

,date,price
0,2021-11-01,3.484
1,2021-10-25,3.476
2,2021-10-18,3.416
3,2021-10-11,3.360
4,2021-10-04,3.285


#### houses prices and rent + personal income

In [15]:
f = open('datasets/house-chain-00.csv', 'rt')
lines = f.readlines()
houses = [[i.strip() for i in l.split(sep = '$') ] for l in lines ]
houses = pd.DataFrame(data=houses, columns=['state', 2000, 1990, 1980, 1970, 1960, 1950, 1940 ])
houses.head()

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/house-chain-00.csv'

#### merging features

In [ ]:
'''
df = pd.merge(pop_vote, nom_gdp, how='inner', left_on=['year','state'], right_on=['year','state'])
df = pd.merge(df, house_vote, how='inner', left_on=['year','state'], right_on=['year','state'])
df.head()
#'''

In [ ]:
'''
df.to_csv('datasets-clean/xxx-final-dataset.csv', index=False)
#'''